In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
import cv2
import numpy as np
import os

In [ ]:
# Set the image dimensions and batch size
img_rows, img_cols, channels = 28, 28, 3
batch_size = 64
latent_dim = 100
num_classes = 2

In [ ]:
from sklearn.datasets import load_files

train_dataset = load_files('/content/drive/MyDrive/data/train')
test_dataset = load_files('/content/drive/MyDrive/data/test')


In [ ]:
train_images = train_dataset.filenames
train_labels = train_dataset.target

test_images = test_dataset.filenames
test_labels = test_dataset.target

In [ ]:
import keras.utils as image
from keras.utils import to_categorical
import numpy as np
# Preprocess the images
train_images = [image.img_to_array(image.load_img(img, target_size=(28, 28))) for img in train_images]
train_images = np.array(train_images)

test_images = [image.img_to_array(image.load_img(img, target_size=(28, 28))) for img in test_images]
test_images = np.array(test_images)
# Preprocess the labels
nClasses=2
train_labels_c = to_categorical(train_labels,num_classes=nClasses)
test_labels_c = to_categorical(test_labels,num_classes=nClasses)

In [ ]:
# Use ImageDataGenerator to read and augment the images
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
# Generate the training and testing datasets
train_generator = train_datagen.flow_from_directory(
    "/content/drive/MyDrive/data/train",
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    class_mode='binary')

test_generator = test_datagen.flow_from_directory(
    "/content/drive/MyDrive/data/test",
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    class_mode='binary')

Found 84 images belonging to 2 classes.
Found 0 images belonging to 0 classes.


In [ ]:
# Build the CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(img_rows, img_cols, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Train the model
model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=10,
    validation_data=test_generator,
    validation_steps=test_generator.samples // batch_size)


<ipython-input-51-fc6408bbde99>:28: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


Epoch 1/10
1/1 [==============================] - 2s 2s/step - loss: 0.6961 - accuracy: 0.5000
Epoch 2/10
1/1 [==============================] - 1s 552ms/step - loss: 0.6831 - accuracy: 0.5938
Epoch 3/10
1/1 [==============================] - 0s 168ms/step - loss: 0.6728 - accuracy: 0.7500
Epoch 4/10
1/1 [==============================] - 1s 513ms/step - loss: 0.6890 - accuracy: 0.4531
Epoch 5/10
1/1 [==============================] - 1s 523ms/step - loss: 0.7041 - accuracy: 0.4531
Epoch 6/10
1/1 [==============================] - 0s 184ms/step - loss: 0.6617 - accuracy: 0.6500
Epoch 7/10
1/1 [==============================] - 1s 554ms/step - loss: 0.7124 - accuracy: 0.4531
Epoch 8/10
1/1 [==============================] - 0s 214ms/step - loss: 0.6695 - accuracy: 0.5500
Epoch 9/10
1/1 [==============================] - 1s 533ms/step - loss: 0.6948 - accuracy: 0.4375
Epoch 10/10
1/1 [==============================] - 0s 199ms/step - loss: 0.7207 - accuracy: 0.4500


In [ ]:
import numpy as np
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout
from tensorflow.keras.layers import BatchNormalization, Activation, ZeroPadding2D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import UpSampling2D, Conv2D
from tensorflow.keras.models import Sequential, Model


img_rows, img_cols, channels = 28, 28, 3
batch_size = 64
latent_dim = 100
num_classes = 2

# Build the generator
generator = Sequential()

generator.add(Dense(128 * 7 * 7, activation="relu", input_dim=latent_dim))
generator.add(Reshape((7, 7, 128)))
generator.add(UpSampling2D())
generator.add(Conv2D(128, kernel_size=3, padding="same"))
generator.add(BatchNormalization(momentum=0.8))
generator.add(Activation("relu"))
generator.add(UpSampling2D())
generator.add(Conv2D(128, kernel_size=3, padding="same"))
generator.add(BatchNormalization(momentum=0.8))
generator.add(Activation("relu"))
generator.add(Conv2D(channels, kernel_size=3, padding="same"))
generator.add(Activation("tanh"))

# Build the discriminator
discriminator = Sequential()

discriminator.add(Conv2D(128, kernel_size=3, strides=2, input_shape=(img_rows, img_cols, channels), padding="same"))
discriminator.add(LeakyReLU(alpha=0.01))
discriminator.add(Dropout(0.25))
discriminator.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
discriminator.add(ZeroPadding2D(padding=((0,1),(0,1))))
discriminator.add(BatchNormalization(momentum=0.8))
discriminator.add(LeakyReLU(alpha=0.01))
discriminator.add(Dropout(0.25))
discriminator.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
discriminator.add(BatchNormalization(momentum=0.8))
discriminator.add(LeakyReLU(alpha=0.01))
discriminator.add(Dropout(0.25))
discriminator.add(Conv2D(128, kernel_size=3, strides=1, padding="same"))
discriminator.add(BatchNormalization(momentum=0.8))
discriminator.add(LeakyReLU(alpha=0.01))



In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
class GeneratorNet(nn.Module):
  def __init__(self, latent_dim, img_size):
    super().__init__()
    # Define the layers of the generator network
    self.x1 = nn.Linear(latent_dim, 128)
    self.x2 = nn.Linear(128, 256)
    self.x3 = nn.Linear(256, img_size)

  def forward(self, x):
    # Define the forward pass of the generator network
    x = self.x1(x)
    x = F.relu(x)
    x = self.x2(x)
    x = F.relu(x)
    x = self.x3(x)
    return x

class DiscriminatorNet(nn.Module):
  def __init__(self, img_size):
    super().__init__()
    # Define the layers of the discriminator network
    self.z1 = nn.Linear(img_size, 128)
    self.z2 = nn.Linear(128, 64)
    self.z3 = nn.Linear(64, 1)

  def forward(self, x):
    # Define the forward pass of the discriminator network
    x = self.z1(x)
    x = F.relu(x)
    x = self.z2(x)
    x = F.relu(x)
    x = self.z3(x)
    return x


In [ ]:
#define the GAN model
class GAN(nn.Module):
  def __init__(self, latent_dim, img_size):
    super().__init__()
    self.generator = GeneratorNet(latent_dim, img_size)
    self.discriminator = DiscriminatorNet(img_size)


In [ ]:
# Compile the GAN model
import tensorflow as tf
import numpy as np
import keras

gan = tf.keras.Sequential()
gan.add(generator)
gan.add(discriminator)

discriminator.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Freeze the weights of the discriminator
discriminator.trainable = False

# Connect the generator and discriminator to create the GAN model
gan = keras.Sequential([generator, discriminator])

gan.compile(optimizer='adam', loss='binary_crossentropy')



In [ ]:
# Train the GAN model
for epoch in range(200):
    # Generate a batch of random noise
    noise = np.random.rand(128, 100)

    # Generate a batch of fake samples
    fake_samples = generator.predict(noise)


4/4 [==============================] - 1s 185ms/step


In [ ]:
gan.save('gan.h5')